In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import random

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation
from torchvision import transforms
#from transformers import DeepLabV3Processor, DeepLabV3ForSemanticSegmentation

from PIL import Image
import evaluate
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from tqdm.notebook import tqdm
import wandb
import segmentation_models_pytorch as smp
#!pip install opencv-python
import cv2

In [2]:
class SemanticSegmentationDataset(Dataset):
    """Image (semantic) segmentation dataset."""

    def __init__(self, root_dir, transform=None, train=True):
        """
        Args:
            root_dir (string): Root directory of the dataset containing the images + annotations.
            image_processor (SegFormerImageProcessor): image processor to prepare images + segmentation maps.
            train (bool): Whether to load "training" or "validation" images + annotations.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.train = train

        sub_path = "training" if self.train else "validation"
        self.img_dir = os.path.join(self.root_dir, sub_path, "images")
        self.ann_dir = os.path.join(self.root_dir, sub_path, "groundtruth")
        self.img_dir = os.path.normpath(self.img_dir)
        self.ann_dir = os.path.normpath(self.ann_dir)
        # read images
        image_file_names = []
        for root, dirs, files in os.walk(self.img_dir):
          if os.path.basename(root) != "images":
            files = [os.path.basename(root) + "/" + f for f in files]
          if os.path.basename(root) == "images":
            image_file_names.extend(files)
        self.images = sorted(image_file_names)
        # read annotations
        annotation_file_names = []
        for root, dirs, files in os.walk(self.ann_dir):
          if os.path.basename(root) != "groundtruth":
            files = [os.path.basename(root) + "/" + f for f in files]
          if os.path.basename(root) == "groundtruth":
            annotation_file_names.extend(files)
        self.annotations = sorted(annotation_file_names)

        #assert len(self.images) == len(self.annotations), "There must be as many images as there are segmentation maps"

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        #print(os.path.join(self.ann_dir, self.annotations[idx]))
        image = Image.open(os.path.join(self.img_dir, self.images[idx])).convert("RGB")
        segmentation_map = Image.open(os.path.join(self.ann_dir, self.annotations[idx]))
        # Convert to tensor
        if self.transform:
          image = self.transform(np.array(image))
          image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)
        
        # Convert segmentation map to tensor, and ensure it's an integer type
        segmentation_map = torch.tensor(np.array(segmentation_map), dtype=torch.long)/250

        return image, segmentation_map

In [4]:
ENCODER = "resnet34"  # Example encoder
ENCODER_WEIGHTS = "imagenet"  # Use ImageNet pretraining weights
preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

model = smp.DeepLabV3Plus(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=2,  # Number of classes (including background)
    activation= None  # or 'sigmoid' for binary segmentation
)
    
model_path = "C:/Users/Qrnqult/Documents/GitHub/ML_AGA/models/finetuned_final_34_DeepLab.pth" 
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
print(f"Model state dict loaded from {model_path}")
root_dir = 'C:/Users/Qrnqult/Documents/GitHub/ML_AGA/data'
valid_dataset = SemanticSegmentationDataset(root_dir=root_dir, train=False, transform=preprocessing_fn)
print("Number of validation examples:", len(valid_dataset))


Model state dict loaded from C:/Users/Qrnqult/Documents/GitHub/ML_AGA/models/finetuned_final_34_DeepLab.pth
Number of validation examples: 20


In [10]:
batch_size = 1
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
all_preds = []
all_labels = []
preds_images = []
labels_images = []
model.eval()  # Set model to evaluation mode
validation_loss = 0.0
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
with torch.no_grad():
    for val_batch in tqdm(valid_dataloader, desc="Validation"):
        # Get the inputs
        image, mask = val_batch
        #print("Shape", image.shape)
        image = image.to(device)
        mask = mask.to(device)
        # Forward pass
        outputs = model(image)
        one_part = outputs[:, 1, :, :]
        val_loss = nn.BCEWithLogitsLoss()(one_part, mask)
        validation_loss += val_loss.item()
        # Evaluate validation metrics
        predicted = outputs.argmax(dim=1)
        preds_images.append(predicted.cpu().numpy())
        labels_images.append(mask.cpu().numpy())
        all_preds.append(predicted.cpu().numpy().flatten())
        all_labels.append(mask.cpu().numpy().flatten())

# Compute overall metrics for validation
avg_val_loss = validation_loss / len(valid_dataloader)
# Compute pixel-wise scores for the entire validation set
all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(np.round(all_labels).astype(int))
preds_images = np.concatenate(preds_images)
labels_images = np.concatenate(np.round(labels_images).astype(int))
assert len(all_preds) == len(all_labels)
pixelwise_f1_val = f1_score(all_labels, all_preds, average="binary")
print("F1 score", pixelwise_f1_val)
pixelwise_recall_val = recall_score(all_labels, all_preds)
print("Recall: ", pixelwise_recall_val)
preds_images = (preds_images * 255).astype(np.uint8)
labels_images = (labels_images * 255).astype(np.uint8)


Validation:   0%|          | 0/20 [00:00<?, ?it/s]

F1 score 0.8608158441952152
Recall:  0.8621327999834925


In [ ]:
im_nbr = 3
predicted_image = Image.fromarray(preds_images[im_nbr], mode='L')
#predicted_image.show()
true_groundtruth_image  = Image.fromarray(labels_images[im_nbr], mode='L')
#true_groundtruth_image.show()
concatenated_image = Image.fromarray(
    np.hstack((np.array(predicted_image), np.array(true_groundtruth_image)))
)
# Show the concatenated image
concatenated_image.show()


In [16]:
path = "C:/Users/Qrnqult/Documents/GitHub/ML_AGA/data/test"
images = sorted(os.listdir(path))
print(images)
outputs = []
outputs2 = []
for im in images:
    print(im)
    # Open the image
    image = Image.open(f"./data/test/{im}/{im}.png").convert("RGB")
    #print(f"{path}/{im}")
    image_prep = preprocessing_fn(np.array(image))
    image_prep = torch.tensor(image_prep, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)
    with torch.no_grad():
        output = model(image_prep)
        output = output.argmax(dim=1)
        output = output.numpy().astype(np.uint8)

    array_scaled = (output[0].squeeze() * 255).astype(np.uint8)
    image = Image.fromarray(array_scaled, mode='L')  # 'L' mode for grayscale
    image.save(f"./data/test/{im}/{im}_DLfinal.png")
    outputs.append(array_scaled)

['test_1', 'test_10', 'test_11', 'test_12', 'test_13', 'test_14', 'test_15', 'test_16', 'test_17', 'test_18', 'test_19', 'test_2', 'test_20', 'test_21', 'test_22', 'test_23', 'test_24', 'test_25', 'test_26', 'test_27', 'test_28', 'test_29', 'test_3', 'test_30', 'test_31', 'test_32', 'test_33', 'test_34', 'test_35', 'test_36', 'test_37', 'test_38', 'test_39', 'test_4', 'test_40', 'test_41', 'test_42', 'test_43', 'test_44', 'test_45', 'test_46', 'test_47', 'test_48', 'test_49', 'test_5', 'test_50', 'test_6', 'test_7', 'test_8', 'test_9']
test_1
test_10
test_11
test_12
test_13
test_14
test_15
test_16
test_17
test_18
test_19
test_2
test_20
test_21
test_22
test_23
test_24
test_25
test_26
test_27
test_28
test_29
test_3
test_30
test_31
test_32
test_33
test_34
test_35
test_36
test_37
test_38
test_39
test_4
test_40
test_41
test_42
test_43
test_44
test_45
test_46
test_47
test_48
test_49
test_5
test_50
test_6
test_7
test_8
test_9


In [ ]:
from mask_to_submission import masks_to_submission
submission_filename = '_pred_final_DeepLabV3Plus_34.csv'
image_filenames = []
for i in range(1, 51):
    image_filename = f"./data/test/test_{i}/test_{i}_DLfinal.png"
    #print(image_filename)
    image_filenames.append(image_filename)
    #print()
masks_to_submission(submission_filename, *image_filenames)

print("Submission file created. Done!")

Submission file created. Done!
